In [89]:
import tensorflow as tf
import numpy as np

In [90]:
#@title Initialisations
batch_size = 8
teacher_epsilons = [1, 1]   # len = num_teachers
num_observations = 2

learn_teacher_epsilons = [0, 0]     # bool 
learned_teacher_epsilons = []    # len = sum(learn_teacher_epsilons)

bounds = [0., 0.]       # threshold on (x,y) distance to target for teacher regularisation

In [91]:
obs = tf.convert_to_tensor(np.random.randn(batch_size, num_observations))

In [92]:
#@title Create and update the teacher epsilons

teacher_epsilons = tf.repeat(tf.convert_to_tensor(
                    [teacher_epsilons], dtype=float), repeats=batch_size, axis=0)
learned_teacher_epsilons = tf.repeat(tf.convert_to_tensor(
                    [learned_teacher_epsilons]), repeats=batch_size, axis=0)
teacher_epsilons = tf.Variable(teacher_epsilons)
learned_teacher_epsilons = tf.Variable(learned_teacher_epsilons)

# Check for dimension mismatch
tf.debugging.assert_equal(
    tf.shape(learned_teacher_epsilons)[1], sum(learn_teacher_epsilons))

In [93]:
count = 0
for i, x in enumerate(learn_teacher_epsilons):
    print(i)
    if x:
        teacher_epsilons = teacher_epsilons[:, i].assign(
            learned_teacher_epsilons[:, count])
        count += 1

0
1


In [94]:
print(teacher_epsilons)

<tf.Variable 'Variable:0' shape=(8, 2) dtype=float32, numpy=
array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)>


In [95]:
obs_bool = tf.cast(tf.where(abs(obs) <= bounds, 0, 1), dtype=tf.float32)

# Find if position is in the small square. If so, use both teachers. 
is_insquare = tf.where(obs_bool[:, 0] + obs_bool[:, 1] == 0, 1, 0)
is_insquare = tf.expand_dims(is_insquare, axis=-1)
is_insquare = tf.broadcast_to(is_insquare, [batch_size, num_observations])
is_insquare = tf.cast(is_insquare, dtype = tf.float32)
obs_bool += is_insquare

teacher_epsilons.assign(obs_bool * teacher_epsilons)

<tf.Variable 'UnreadVariable' shape=(8, 2) dtype=float32, numpy=
array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)>

In [96]:
print(obs)

tf.Tensor(
[[-0.58573167  1.06876996]
 [-0.07761734  0.50615729]
 [-1.83031819 -0.1479727 ]
 [-0.15158393 -0.24046586]
 [ 0.74955873  0.2486711 ]
 [-0.05471362 -0.09137543]
 [-1.67013937 -1.08629146]
 [-0.47898374 -0.94952435]], shape=(8, 2), dtype=float64)


In [97]:
# Change epsilon based on angle to target 
# teacher1 = go_horizontal teacher --> go to x=0
# teacher2 = go_vertical teacher --> go to y=0

# compute angle from origin 

# vector of distance of each observation from origin
r = np.maximum(np.linalg.norm(obs, axis=1), 1e-4)

# prefactors to multiply the teacher epsilons with, element wise
# cosine of angle w.r.t. horizontal and vertical axes, respectively 
teacher_fac = np.abs(obs/r[:, None])

# weight teacher epsilon by angle to origin 
teacher_epsilons = np.multiply(teacher_epsilons, teacher_fac)

In [98]:
teacher_epsilons

array([[0.48060046, 0.87693968],
       [0.1515745 , 0.98844584],
       [0.99674795, 0.08058243],
       [0.53326538, 0.84594801],
       [0.94913128, 0.31488062],
       [0.5137248 , 0.85795503],
       [0.83828283, 0.54523563],
       [0.45038639, 0.89283375]])

In [99]:
obs

<tf.Tensor: shape=(8, 2), dtype=float64, numpy=
array([[-0.58573167,  1.06876996],
       [-0.07761734,  0.50615729],
       [-1.83031819, -0.1479727 ],
       [-0.15158393, -0.24046586],
       [ 0.74955873,  0.2486711 ],
       [-0.05471362, -0.09137543],
       [-1.67013937, -1.08629146],
       [-0.47898374, -0.94952435]])>